In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('Train.csv')
df.head()

,Inv_Id,Vendor_Code,GL_Code,Inv_Amt,Item_Description,Product_Category
0,15001,VENDOR-1676,GL-6100410,83.24,Artworking/Typesetting Production Jun 2009 Cha...,CLASS-1963
1,15002,VENDOR-1883,GL-2182000,51.18,Auto Leasing Corporate Services Corning Inc /N...,CLASS-1250
2,15004,VENDOR-1999,GL-6050100,79.02,Store Management Lease/Rent Deltona Corp Real ...,CLASS-1274
3,15005,VENDOR-1771,GL-6101400,48.50,Store Construction General Requirements Coloni...,CLASS-1522
4,15006,VENDOR-1331,GL-2182000,63.35,Jul 2015 Aydin Corp Contingent Labor/Temp Labo...,CLASS-1376


In [3]:
df['Product_Category'] = df.Product_Category.astype('category')

In [4]:
cols = ['Inv_Amt', 'Item_Description']
X = df[cols]
print(type(X))
print(X.shape)
y = df['Product_Category']
print(type(y))

<class 'pandas.core.frame.DataFrame'>
(5566, 2)
<class 'pandas.core.series.Series'>


In [5]:
numeric_cols = ['Inv_Amt']
label = ['Product_Category']

In [6]:
def combine_text_cols(df, to_drop=numeric_cols+label):
    to_drop = set(to_drop) & set(df.columns.tolist())
    text_data = df.drop(to_drop, axis=1)
    return text_data.apply(lambda x:" ".join(x), axis=1)

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import Imputer
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.preprocessing import FunctionTransformer, MaxAbsScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion

In [8]:
X_train, x_test, y_train, y_test = train_test_split(X, pd.get_dummies(y), test_size=0.10,  random_state=2)

In [9]:
get_text_data = FunctionTransformer(combine_text_cols, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[numeric_cols], validate=False)
tokens_alphanumeric = '[A-Za-z0-9]+(?=\\s+)'

In [10]:
chik=300

In [11]:
pl = Pipeline([('union', FeatureUnion(transformer_list=[('numeric_features', Pipeline([('selector',
                                                                                        get_numeric_data),
                                                                                       ('imputer', Imputer())])),
                                                        ('text_feature', Pipeline([('selector', get_text_data),
                                                                                   ('vec', HashingVectorizer(norm=None, non_negative=True, token_pattern=tokens_alphanumeric,
                                                                                                             ngram_range=(1,2))),
                                                                                   ('dim_red', SelectKBest(chi2, chik))])
                                                        )])),
               ('int', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),
               ('scale', MaxAbsScaler()),
               ('clf', OneVsRestClassifier(LogisticRegression())
               )])

C:\Users\HP\Anaconda2\envs\data_analysis\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [12]:
pl.fit(X_train, y_train)

C:\Users\HP\Anaconda2\envs\data_analysis\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\HP\Anaconda2\envs\data_analysis\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
C:\Users\HP\Anaconda2\envs\data_analysis\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('numeric_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function <lambda> at 0x0000020FB0C071E0>, inv_kw_args=None,
          inverse_func=None...te=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None))])

In [ ]:
y_pred = pl.predict(x_test)
y_proba = pl.predict_proba(x_test)

C:\Users\HP\Anaconda2\envs\data_analysis\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [ ]:
from sklearn.metrics import accuracy_score, fbeta_score, mean_squared_error
acc_y = accuracy_score(y_test, y_pred)
f1_y = fbeta_score(y_test, y_pred, beta=1, average='micro')
mse = mean_squared_error(y_test, y_pred)
print('accuracy', acc_y)
print('f1 score', f1_y)
print('MSE', mse)

In [33]:
df_t = pd.read_csv('Test.csv')
df_t.head()

,Inv_Id,Vendor_Code,GL_Code,Inv_Amt,Item_Description
0,15003,VENDOR-2513,GL-6050310,56.13,Travel and Entertainment Miscellaneous Company...
1,15008,VENDOR-1044,GL-6101400,96.56,Final Site Clean Up Store Construction Advance...
2,15013,VENDOR-1254,GL-6101400,55.93,Arabian American Development Co Final Site Cle...
3,15019,VENDOR-1331,GL-2182000,32.62,Corporate Services Contingent Labor/Temp Labor...
4,15020,VENDOR-2513,GL-6050310,25.81,Fortune National Corp Miscellaneous Company Ca...


In [34]:
sub =  pd.read_csv('sample_submission.csv')
sub.head()

,Inv_Id,Product_Category
0,1,CLASS-784
1,2,CLASS-784
2,3,CLASS-784
3,4,CLASS-784
4,5,CLASS-784


In [35]:
ID_col = df_t['Inv_Id'].to_list()

In [36]:
test = df_t[cols]

In [37]:
t_y = pl.predict_proba(test)

C:\Users\HP\Anaconda2\envs\data_analysis\lib\site-packages\sklearn\feature_extraction\hashing.py:102: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [38]:
y_cols = pd.get_dummies(y).columns.to_list()
y_cols

['CLASS-1248',
 'CLASS-1249',
 'CLASS-1250',
 'CLASS-1274',
 'CLASS-1294',
 'CLASS-1309',
 'CLASS-1322',
 'CLASS-1376',
 'CLASS-1429',
 'CLASS-1477',
 'CLASS-1522',
 'CLASS-1567',
 'CLASS-1652',
 'CLASS-1688',
 'CLASS-1721',
 'CLASS-1758',
 'CLASS-1770',
 'CLASS-1805',
 'CLASS-1828',
 'CLASS-1838',
 'CLASS-1850',
 'CLASS-1867',
 'CLASS-1870',
 'CLASS-1919',
 'CLASS-1957',
 'CLASS-1963',
 'CLASS-1964',
 'CLASS-1983',
 'CLASS-2003',
 'CLASS-2015',
 'CLASS-2038',
 'CLASS-2112',
 'CLASS-2141',
 'CLASS-2146',
 'CLASS-2152',
 'CLASS-2241']

In [39]:
col_index = np.argmax(t_y, axis=1)
col_index

array([15, 10, 10, ...,  2, 25, 26], dtype=int64)

In [41]:
target = [y_cols[ind] for ind in col_index]


In [42]:
print(len(target))
print(len(ID_col))

2446
2446


In [43]:
dict_t = {'Inv_Id':ID_col,
         'Product_Category':target}
df_t = pd.DataFrame(dict_t)

In [44]:
df_t.to_csv('predictions.csv', index=False)